In [2]:
import pandas as pd
import numpy as np

In [4]:
#There are 640002 rows of data
nrows = np.arange(640002)
nrows = np.delete(nrows, np.arange(0, 640002, 2))
data = pd.read_csv("../Data/test_data.csv", skiprows = nrows)

In [5]:
print(data.columns.values)

['MSG ID' 'TimeHigh' 'TimeLow' 'CDP Control' 'CDP Status' 'IM Gap' 'CMD1'
 'CMD2' 'STS1' 'STS2' ' DATAWRD 00' ' DATAWRD 01' ' DATAWRD 02'
 ' DATAWRD 03' ' DATAWRD 04' ' DATAWRD 05' ' DATAWRD 06' ' DATAWRD 07'
 ' DATAWRD 08' ' DATAWRD 09' ' DATAWRD 10' ' DATAWRD 11' ' DATAWRD 12'
 ' DATAWRD 13' ' DATAWRD 14' ' DATAWRD 15' ' DATAWRD 16' ' DATAWRD 17'
 ' DATAWRD 18' ' DATAWRD 19' ' DATAWRD 20' ' DATAWRD 21' ' DATAWRD 22'
 ' DATAWRD 23' ' DATAWRD 24' ' DATAWRD 25' ' DATAWRD 26' ' DATAWRD 27'
 ' DATAWRD 28' ' DATAWRD 29' ' DATAWRD 30' ' DATAWRD 31']


In [6]:
CDP_Status = data['CDP Status'][0]
CDP_Status = CDP_Status[2:]
IM_Gap = data['IM Gap'][0]
IM_Gap = IM_Gap[2:]

int_CDP = int(CDP_Status, 16)
int_IM = int(IM_Gap, 16)
bin_CDP = bin(int(CDP_Status, 16)).zfill(8)[2:]
bin_IM = bin(int(IM_Gap, 16)).zfill(8)[2:]

#sub-selecting columns to look at specific attributes
data1 = data[['CMD1', 'STS1', ' DATAWRD 00', ' DATAWRD 01', ' DATAWRD 02']]

data2 = data1.applymap(lambda x: bin(int(x, 16)).zfill(20))
data2 = data2.applymap(lambda x: x.replace("b", "0", 1))
data2 = data2.applymap(lambda x: x[2:] if len(x) > 20 else x)

data2

,CMD1,STS1,DATAWRD 00,DATAWRD 01,DATAWRD 02
0,00000000110000100010,00000000100000000000,00000000000000000000,00000000000000000000,11111111111111111111111111111111
1,00000000110001000100,00000000100000000000,00000100000001101001,00000000000000000000,00000000000000000000
2,00000000110001100100,00000000100000000000,00000011111111010001,00001111101110001001,00001110000000000000
3,00000000110010000100,00000000100000000000,00000011111111011001,00000111111111110010,00000110000000000000
4,00000001010000100010,00000001000000000000,00000000000000000000,00000000000000000000,11111111111111111111111111111111
...,...,...,...,...,...
319996,00000010010001100100,00000010000000000000,00000011111111011110,00000000000000000011,00000000000000000000
319997,00000010110000100100,00000010100000000000,00000100000001000100,00000101101000101000,00000110000000000000
319998,00000010110001000100,00000010100000000000,00001100000001010010,00000111001100000101,00000110000000000000
319999,00000010110001100100,00000010100000000000,00000100000011001001,00000001100111111011,00000110000000000000


In [7]:
#working on CMD1 column
command_column_names = ["Sync Waveform", "Remote Terminal Address", "T/R", "Subaddress Mode", "DW-Count/Mode Word", "Parity"]
command_df = pd.DataFrame(columns = command_column_names)

command_df["Sync Waveform"] = data2['CMD1'].apply(lambda row: row[0:3])
command_df["Remote Terminal Address"] = data2['CMD1'].apply(lambda row: row[3:8])
command_df["T/R"] = data2['CMD1'].apply(lambda row: row[8:9])
command_df["Subaddress Mode"] = data2['CMD1'].apply(lambda row: row[9:14])
command_df["DW-Count/Mode Word"] = data2['CMD1'].apply(lambda row: row[14:19])
command_df["Parity"] = data2['CMD1'].apply(lambda row: row[19:])

command_df = command_df.applymap(lambda x: (np.fromstring(x, np.int8)-48))
command_df

/home/aviraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


,Sync Waveform,Remote Terminal Address,T/R,Subaddress Mode,DW-Count/Mode Word,Parity
0,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],"[1, 0, 0, 0, 0]","[1, 0, 0, 0, 1]",[0]
1,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],"[1, 0, 0, 0, 1]","[0, 0, 0, 1, 0]",[0]
2,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],"[1, 0, 0, 0, 1]","[1, 0, 0, 1, 0]",[0]
3,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],"[1, 0, 0, 1, 0]","[0, 0, 0, 1, 0]",[0]
4,"[0, 0, 0]","[0, 0, 0, 0, 1]",[0],"[1, 0, 0, 0, 0]","[1, 0, 0, 0, 1]",[0]
...,...,...,...,...,...,...
319996,"[0, 0, 0]","[0, 0, 0, 1, 0]",[0],"[1, 0, 0, 0, 1]","[1, 0, 0, 1, 0]",[0]
319997,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],"[1, 0, 0, 0, 0]","[1, 0, 0, 1, 0]",[0]
319998,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],"[1, 0, 0, 0, 1]","[0, 0, 0, 1, 0]",[0]
319999,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],"[1, 0, 0, 0, 1]","[1, 0, 0, 1, 0]",[0]


In [8]:
#Work on STS1 column
status_column_names = ["Sync Waveform", "Remote Terminal Address", "MError", "Instrumentation", "Service Request", "Reserved", \
                       "Broadcast", "Busy", "Subsystem Flag", "D Bus Control", "Terminal Flag", "Parity"]
status_df = pd.DataFrame(columns = status_column_names) 

status_df["Sync Waveform"] = data2['STS1'].apply(lambda row: row[0:3])
status_df["Remote Terminal Address"] = data2['STS1'].apply(lambda row: row[3:8])
status_df["MError"] = data2['STS1'].apply(lambda row: row[8:9])
status_df["Instrumentation"] = data2['STS1'].apply(lambda row: row[9:10])
status_df["Service Request"] = data2['STS1'].apply(lambda row: row[10:11])
status_df["Reserved"] = data2['STS1'].apply(lambda row: row[11:14])
status_df["Broadcast"] = data2['STS1'].apply(lambda row: row[14:15])
status_df["Busy"] = data2['STS1'].apply(lambda row: row[15:16])
status_df["Subsystem Flag"] = data2['STS1'].apply(lambda row: row[16:17])
status_df["D Bus Control"] = data2['STS1'].apply(lambda row: row[17:18])
status_df["Terminal Flag"] = data2['STS1'].apply(lambda row: row[18:19])
status_df["Parity"] = data2['STS1'].apply(lambda row: row[19:])


status_df = status_df.applymap(lambda x: (np.fromstring(x, np.int8)-48))
status_df

/home/aviraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


,Sync Waveform,Remote Terminal Address,MError,Instrumentation,Service Request,Reserved,Broadcast,Busy,Subsystem Flag,D Bus Control,Terminal Flag,Parity
0,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
1,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
2,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
3,"[0, 0, 0]","[0, 0, 0, 0, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
4,"[0, 0, 0]","[0, 0, 0, 0, 1]",[0],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...
319996,"[0, 0, 0]","[0, 0, 0, 1, 0]",[0],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
319997,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
319998,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]
319999,"[0, 0, 0]","[0, 0, 0, 1, 0]",[1],[0],[0],"[0, 0, 0]",[0],[0],[0],[0],[0],[0]


In [9]:
#Working on Dataword00 column
data_column_names = ["Sync Waveform", "Data", "Parity"]
data00_df = pd.DataFrame(columns = data_column_names)

data00_df[data_column_names[0]] = data2[' DATAWRD 00'].apply(lambda row: row[0:3])
data00_df[data_column_names[1]] = data2[' DATAWRD 00'].apply(lambda row: row[3:19])
data00_df[data_column_names[2]] = data2[' DATAWRD 00'].apply(lambda row: row[19:])

data00_df = data00_df.applymap(lambda x: (np.fromstring(x, np.int8)-48))
data00_df

/home/aviraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':


,Sync Waveform,Data,Parity
0,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
1,"[0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]",[1]
2,"[0, 0, 0]","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]",[1]
3,"[0, 0, 0]","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0]",[1]
4,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
...,...,...,...
319996,"[0, 0, 0]","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",[0]
319997,"[0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]",[0]
319998,"[0, 0, 0]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]",[0]
319999,"[0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]",[1]


In [10]:
#Working on Dataword01 column
data01_df = pd.DataFrame(columns = data_column_names)

data01_df[data_column_names[0]] = data2[' DATAWRD 01'].apply(lambda row: row[0:3])
data01_df[data_column_names[1]] = data2[' DATAWRD 01'].apply(lambda row: row[3:19])
data01_df[data_column_names[2]] = data2[' DATAWRD 01'].apply(lambda row: row[19:])

data01_df = data01_df.applymap(lambda x: (np.fromstring(x, np.int8)-48))
data01_df

/home/aviraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


,Sync Waveform,Data,Parity
0,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
1,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
2,"[0, 0, 0]","[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0]",[1]
3,"[0, 0, 0]","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]",[0]
4,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
...,...,...,...
319996,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",[1]
319997,"[0, 0, 0]","[0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]",[0]
319998,"[0, 0, 0]","[0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]",[1]
319999,"[0, 0, 0]","[0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1]",[1]


In [11]:
#Working on Dataword02 column
data02_df = pd.DataFrame(columns = data_column_names)

data02_df[data_column_names[0]] = data2[' DATAWRD 02'].apply(lambda row: row[0:3])
data02_df[data_column_names[1]] = data2[' DATAWRD 02'].apply(lambda row: row[3:19])
data02_df[data_column_names[2]] = data2[' DATAWRD 02'].apply(lambda row: row[19:])

data02_df = data02_df.applymap(lambda x: (np.fromstring(x, np.int8)-48))
data02_df

/home/aviraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


,Sync Waveform,Data,Parity
0,"[1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
2,"[0, 0, 0]","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
3,"[0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
4,"[1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...
319996,"[0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
319997,"[0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
319998,"[0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]
319999,"[0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0]


In [12]:
preprocessed_message_df = pd.concat([status_df, command_df, data00_df,data01_df, data02_df],axis=1)

preprocessed_message_df.to_pickle("preprocessed")

## Aggregated by terminal analysis

In [18]:
preprocessed_message_df = pd.read_csv("../Data/preprocessed.csv")

In [19]:
preprocessed_message_df.columns

Index(['Unnamed: 0', 'Sync Waveform', 'Remote Terminal Address', 'MError',
       'Instrumentation', 'Service Request', 'Reserved', 'Broadcast', 'Busy',
       'Subsystem Flag', 'D Bus Control', 'Terminal Flag', 'Parity',
       'Sync Waveform.1', 'Remote Terminal Address.1', 'T/R',
       'Subaddress Mode', 'DW-Count/Mode Word', 'Parity.1', 'Sync Waveform.2',
       'Data', 'Parity.2', 'Sync Waveform.3', 'Data.1', 'Parity.3',
       'Sync Waveform.4', 'Data.2', 'Parity.4'],
      dtype='object')

In [20]:
preprocessed_message_df.tail(3)

,Unnamed: 0,Sync Waveform,Remote Terminal Address,MError,Instrumentation,Service Request,Reserved,Broadcast,Busy,Subsystem Flag,...,Parity.1,Sync Waveform.2,Data,Parity.2,Sync Waveform.3,Data.1,Parity.3,Sync Waveform.4,Data.2,Parity.4
319998,319998,[0 0 0],[0 0 0 1 0],[1],[0],[0],[0 0 0],[0],[0],[0],...,[0],[0 0 0],[0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1],[0],[0 0 0],[0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0],[1],[0 0 0],[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0],[0]
319999,319999,[0 0 0],[0 0 0 1 0],[1],[0],[0],[0 0 0],[0],[0],[0],...,[0],[0 0 0],[0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0],[1],[0 0 0],[0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1],[1],[0 0 0],[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0],[0]
320000,320000,[0 0 0],[0 0 0 0 0],[1],[0],[0],[0 0 0],[0],[0],[0],...,[0],[0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0],[0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0],[1 1 1],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1],[1 1 1 1 1 1 1 1 1 1 1 1 1]


In [21]:
command_df["Remote Terminal Address"] = command_df["Remote Terminal Address"].apply(lambda row: str(row))
command_df["Parity"] = command_df["Parity"].apply(lambda row: str(row))
command_df["T/R"] = command_df["T/R"].apply(lambda row: str(row))
command_df["DW-Count/Mode Word"] = command_df["DW-Count/Mode Word"].apply(lambda row: str(row))
command_df["Subaddress Mode"] = command_df["Subaddress Mode"].apply(lambda row: str(row))

In [22]:
command_df.groupby(['Remote Terminal Address', 'T/R', 'Subaddress Mode']).size()

Remote Terminal Address  T/R  Subaddress Mode
[0 0 0 0 0]              [1]  [1 0 0 0 0]        16001
                              [1 0 0 0 1]        32000
                              [1 0 0 1 0]        16000
[0 0 0 0 1]              [0]  [1 0 0 0 0]        16000
                              [1 0 0 0 1]        16000
                         [1]  [1 0 0 0 0]        16000
                              [1 0 0 0 1]        32000
                              [1 0 0 1 0]        32000
                              [1 0 0 1 1]        32000
                              [1 0 1 0 0]        16000
[0 0 0 1 0]              [0]  [1 0 0 0 0]        16000
                              [1 0 0 0 1]        32000
                         [1]  [1 0 0 0 0]        16000
                              [1 0 0 0 1]        32000
dtype: int64

In [23]:
command_df[command_df["Remote Terminal Address"].str.endswith("1 0]")]

,Sync Waveform,Remote Terminal Address,T/R,Subaddress Mode,DW-Count/Mode Word,Parity
14,"[0, 0, 0]",[0 0 0 1 0],[0],[1 0 0 0 0],[1 0 0 1 0],[0]
15,"[0, 0, 0]",[0 0 0 1 0],[0],[1 0 0 0 1],[0 0 0 1 0],[0]
16,"[0, 0, 0]",[0 0 0 1 0],[0],[1 0 0 0 1],[1 0 0 1 0],[0]
17,"[0, 0, 0]",[0 0 0 1 0],[1],[1 0 0 0 0],[1 0 0 1 0],[0]
18,"[0, 0, 0]",[0 0 0 1 0],[1],[1 0 0 0 1],[0 0 0 1 0],[0]
...,...,...,...,...,...,...
319995,"[0, 0, 0]",[0 0 0 1 0],[0],[1 0 0 0 1],[0 0 0 1 0],[0]
319996,"[0, 0, 0]",[0 0 0 1 0],[0],[1 0 0 0 1],[1 0 0 1 0],[0]
319997,"[0, 0, 0]",[0 0 0 1 0],[1],[1 0 0 0 0],[1 0 0 1 0],[0]
319998,"[0, 0, 0]",[0 0 0 1 0],[1],[1 0 0 0 1],[0 0 0 1 0],[0]


In [24]:
#skip this block to avoid string conversion
#trying to group by address and count errors
status_df["Remote Terminal Address"] = status_df["Remote Terminal Address"].apply(lambda row: str(row))
status_df["MError"] = status_df["MError"].apply(lambda row: str(row))

In [25]:
status_df.groupby(['Remote Terminal Address', 'MError']).size()

Remote Terminal Address  MError
[0 0 0 0 0]              [1]        64001
[0 0 0 0 1]              [0]        32000
                         [1]       128000
[0 0 0 1 0]              [0]        48000
                         [1]        48000
dtype: int64

In [26]:
preprocessed_message_df.columns

Index(['Unnamed: 0', 'Sync Waveform', 'Remote Terminal Address', 'MError',
       'Instrumentation', 'Service Request', 'Reserved', 'Broadcast', 'Busy',
       'Subsystem Flag', 'D Bus Control', 'Terminal Flag', 'Parity',
       'Sync Waveform.1', 'Remote Terminal Address.1', 'T/R',
       'Subaddress Mode', 'DW-Count/Mode Word', 'Parity.1', 'Sync Waveform.2',
       'Data', 'Parity.2', 'Sync Waveform.3', 'Data.1', 'Parity.3',
       'Sync Waveform.4', 'Data.2', 'Parity.4'],
      dtype='object')

In [ ]:
from sklearn.metrics import hamming_loss

